in this notebook I am going to save the sources npred in to datasets, so it can be summed later

In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
import matplotlib.pyplot as plt
import numpy as np
from regions import CircleSkyRegion
import yaml

from gammapy.catalog import CATALOG_REGISTRY
from gammapy.data import EventList
from gammapy.datasets import MapDataset
from gammapy.irf import PSFMap, EDispKernelMap
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    PointSpatialModel,
    SkyModel,
    TemplateSpatialModel,
    TemplateSpectralModel,
    PowerLawNormSpectralModel,
    Models,
    SuperExpCutoffPowerLaw4FGLSpectralModel,
    create_fermi_isotropic_diffuse_model,
    ExpCutoffPowerLawSpectralModel,
)
from gammapy.modeling import Fit

### Opening one dataset for getting geom

In [2]:
sig_threshold_soft = 5

dataset_idx = 1
n_bin_met = 0

dataset_names = ['low_energy', 'med_energy', 'hi_energy']
main_path = '/home/woody/caph/mppi062h/woody_output/final/fermilat/'

folder = f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met'
dataset = MapDataset.read(f'{folder}/fermi_dataset.fits')

### getting the sources from 4FGL for this geom

In [3]:
geom = dataset.counts.geom
fgl = CATALOG_REGISTRY.get_cls("4fgl")()
inside_geom = (geom.drop('energy')).contains(fgl.positions)
idx = np.where(inside_geom)[0]

### organizing the sources by significance
sig_list = []
name_list = []
for i in idx: 
    sig = float(fgl[i].info('more').split()[11])   
    if sig > sig_threshold_soft:
        sig_list.append(sig)
        name_list.append(fgl[i].name)

In [4]:
sig_sorted=np.sort(sig_list)[::-1]

with open('source_list.txt', 'w+') as file:
    for sig in sig_sorted:
        i = np.where(sig_list == sig)[0][0]
        file.write(f'{name_list[i]}\n')

In [19]:
import os
main_path = '/home/woody/caph/mppi062h/woody_output/final/fermilat/'
N_bin_met = 8

for dataset_idx, dataset_name in enumerate(dataset_names):
    if dataset_idx != 0:
        for n_bin_met in range(N_bin_met-1):
            folder = f'{main_path}/{dataset_names[dataset_idx]}/{n_bin_met}_bin_met/sources_dataset/'
            if not os.path.exists(folder):
                os.makedirs(folder)

### putting to run in the cluster

In [1]:
with open('source_list.txt') as f:
    source_list = f.readlines()

In [3]:
'''this script is to run in parallel in ecap hpc cluster'''

import numpy as np
import yaml
import shutil
import os

# some general configuration for running in ECAP cluster
walltime = '00:30:00'
dataset_names = ['low_energy', 'med_energy', 'hi_energy']
N_bin_met = 8

fin = open(f'creating_source_dataset.py', 'rt')
script_original = fin.read()
fin.close()

fin_sub = open(f'to_submit.sh', 'rt')
sub_original = fin_sub.read()
fin_sub.close()

for source_name in source_list[10:50]:
    source_name = source_name[:-1]
    print(source_name)
    for dataset_idx, dataset_name in enumerate(dataset_names):
        if dataset_idx != 0:
            for n_bin_met in range(N_bin_met-1):
                script=script_original.replace('dataset_idx = 1\n', f'dataset_idx = {dataset_idx}\n')
                script=script.replace("n_bin_met = 0\n", f'n_bin_met = {n_bin_met}\n')
                script=script.replace("source_name= 'kaori'\n", f'source_name = "{source_name}"\n')
                
                output_name = f'to_del/{source_name[5:]}_{dataset_idx}_{n_bin_met}.py'
                script_output_path = f'scripts_parallel/{output_name}'
                fout=open(script_output_path, 'w+')
                fout.write(script)
                fout.close()

                script=sub_original.replace('#PBS -l nodes=1:ppn=4,walltime=02:00:00\n', f'#PBS -l nodes=1:ppn=4,walltime={walltime}\n')
                script=script.replace('#PBS -N fermi_dataset_parallel\n', f'#PBS -N fermi_{dataset_idx}_{n_bin_met}\n')
                script=script.replace('python create_dataset.py\n', f'python {output_name}')
                fout=open(f'scripts_parallel/to_submit_parallel.sh', 'w+')
                fout.write(script)
                fout.close()

                os.system(f"qsub -o scripts_parallel/fermi_{dataset_idx}_{n_bin_met}.log -e scripts_parallel/fermi_{dataset_idx}_{n_bin_met}_error.log scripts_parallel/to_submit_parallel.sh")

4FGL J1802.4-3041
8163042.wadm1.rrze.uni-erlangen.de
8163043.wadm1.rrze.uni-erlangen.de
8163044.wadm1.rrze.uni-erlangen.de
8163045.wadm1.rrze.uni-erlangen.de
8163046.wadm1.rrze.uni-erlangen.de
8163047.wadm1.rrze.uni-erlangen.de
8163048.wadm1.rrze.uni-erlangen.de
8163049.wadm1.rrze.uni-erlangen.de
8163050.wadm1.rrze.uni-erlangen.de
8163051.wadm1.rrze.uni-erlangen.de
8163052.wadm1.rrze.uni-erlangen.de
8163053.wadm1.rrze.uni-erlangen.de
8163054.wadm1.rrze.uni-erlangen.de
8163055.wadm1.rrze.uni-erlangen.de
4FGL J1748.3-2906
8163056.wadm1.rrze.uni-erlangen.de
8163057.wadm1.rrze.uni-erlangen.de
8163058.wadm1.rrze.uni-erlangen.de
8163059.wadm1.rrze.uni-erlangen.de
8163060.wadm1.rrze.uni-erlangen.de
8163061.wadm1.rrze.uni-erlangen.de
8163062.wadm1.rrze.uni-erlangen.de
8163063.wadm1.rrze.uni-erlangen.de
8163064.wadm1.rrze.uni-erlangen.de
8163065.wadm1.rrze.uni-erlangen.de
8163066.wadm1.rrze.uni-erlangen.de
8163067.wadm1.rrze.uni-erlangen.de
8163068.wadm1.rrze.uni-erlangen.de
8163069.wadm1.rrze.